![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Proyecto 1 - Predicción de precios de vehículos usados

En este proyecto podrán poner en práctica sus conocimientos sobre modelos predictivos basados en árboles y ensambles, y sobre la disponibilización de modelos. Para su desasrrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 1: Predicción de precios de vehículos usados".

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 4. Sin embargo, es importante que avancen en la semana 3 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 4, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/t/4bd64c1deb3c4ffb8ca7ff93c1f1497d).

## Datos para la predicción de precios de vehículos usados

En este proyecto se usará el conjunto de datos de Car Listings de Kaggle, donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como: año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

## Ejemplo predicción conjunto de test para envío a Kaggle

En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
import numpy as np

In [3]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/dataTrain_carListings.zip')
dataTesting = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/dataTest_carListings.zip', index_col=0)

In [4]:
# Visualización datos de entrenamiento
dataTraining.head()

,Price,Year,Mileage,State,Make,Model
0,34995,2017,9913,FL,Jeep,Wrangler
1,37895,2015,20578,OH,Chevrolet,Tahoe4WD
2,18430,2012,83716,TX,BMW,X5AWD
3,24681,2014,28729,OH,Cadillac,SRXLuxury
4,26998,2013,64032,CO,Jeep,Wrangler


In [5]:
# Visualización datos de test
dataTesting.head()

,Year,Mileage,State,Make,Model
ID,,,,,
0,2014,31909,MD,Nissan,MuranoAWD
1,2017,5362,FL,Jeep,Wrangler
2,2014,50300,OH,Ford,FlexLimited
3,2004,132160,WA,BMW,5
4,2015,25226,MA,Jeep,Grand


In [6]:
#Quitamos la variable Make porque ya model no está dando la especificación del carro
dataTraining = dataTraining.drop(['Make'], axis=1)
dataTesting = dataTesting.drop(['Make'], axis=1)

#Codificamos la variable state y la variable model
dataTraining['State'] = pd.factorize(dataTraining.State)[0]
dataTraining['Model'] = pd.factorize(dataTraining.Model)[0]
dataTesting['State'] = pd.factorize(dataTesting.State)[0]
dataTesting['Model'] = pd.factorize(dataTesting.Model)[0]

#Visualizamos la database de training
dataTraining.head()

,Price,Year,Mileage,State,Model
0,34995,2017,9913,0,0
1,37895,2015,20578,1,1
2,18430,2012,83716,2,2
3,24681,2014,28729,1,3
4,26998,2013,64032,3,0


In [7]:
#Visualizamos la database de testing
dataTesting.head()

,Year,Mileage,State,Model
ID,,,,
0,2014,31909,0,0
1,2017,5362,1,1
2,2014,50300,2,2
3,2004,132160,3,3
4,2015,25226,4,4


In [8]:
# Selección de variables predictoras
X_train = dataTraining.iloc[:, [1,2,3,4]].astype(np.float)
X_test = dataTesting.iloc[:, [0,1,2,3]].astype(np.float)

# Definición variable de interés binaria (y)
y_train = dataTraining.iloc[:, 0].astype(np.float)

In [9]:
#from sklearn.model_selection import train_test_split

# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test usandola función train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=40)

In [10]:
# Se calibra el modelo XGB con 'Grid Search Cross-Validation'
from xgboost import XGBRegressor, plot_importance
from sklearn.model_selection import GridSearchCV

# Definir la cuadrícula de parámetros
param_grid = {
    'gamma': [0, 0.1, 0.5],
    'learning_rate': [0.01, 0.1, 0.5],
    'colsample_bytree': [0.5, 0.7, 1],
    'n_estimators': [100, 200, 300],
}

# Se aplica la estrategia GridSearch
grid_search = GridSearchCV(estimator=XGBRegressor(), param_grid=param_grid, cv=6, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Obtener los mejores parámetros encontrados
print("Mejores parametros encontrados: ", grid_search.best_params_)
print("RMSE más bajo encontrado: ", np.sqrt(np.abs(grid_search.best_score_)))


Mejores parametros encontrados:  {'colsample_bytree': 1, 'gamma': 0, 'learning_rate': 0.5, 'n_estimators': 300}
RMSE más bajo encontrado:  3652.214112502496


In [13]:
# Calibración modelo XGB con los mejores parámetros
xgb_reg_calibrado = XGBRegressor(gamma=0, learning_rate=0.5, colsample_bytree=1, n_estimators=300)

# Entrenamiento (fit) y desempeño del modelo XGBClassifier Calibrado
xgb_reg_calibrado.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.5, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=300, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [16]:
# Guardar predicciones en formato exigido en la competencia de kaggle
y_pred = pd.DataFrame(xgb_reg_calibrado.predict(X_test), index=X_test.index, columns=['Price'])
y_pred.to_csv('test_submission_G26.csv', index_label='ID')
y_pred

,Price
ID,
0,31994.548828
1,49987.886719
2,25524.439453
3,9355.213867
4,29221.847656
...,...
99995,22341.382812
99996,31235.216797
99997,30991.939453


In [ ]:
# Preprocesamiento de datos
X_train = dataTraining[['Year', 'Mileage', 'State', 'Make', 'Model']]
y_train = dataTraining['Price']
X_test = dataTesting[['Year', 'Mileage', 'State', 'Make', 'Model']]
# Codificación One-Hot para las variables categóricas
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

In [ ]:
# eliminar esta marca que no esta en la base de training

X_train = X_train.drop(['Make_Freightliner'], axis=1)

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Crear el modelo de Random Forest con 100 árboles
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Entrenar el modelo
rf_model.fit(X_train, y_train)

# Hacer las predicciones en el conjunto de prueba
y_pred = pd.DataFrame(rf_model.predict(X_test), index=X_test.index, columns=['Price'])

# exportacion

y_pred.to_csv('test_submission_G26.csv', index_label='ID')

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

# Crear el modelo de Random Forest
rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)

# Definir los parámetros que queremos ajustar
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10]
}

# Realizar la búsqueda de parámetros utilizando validación cruzada
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Obtener los mejores parámetros y el mejor score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Entrenar el modelo con los mejores parámetros
rf_best = RandomForestRegressor(**best_params, random_state=42)
rf_best.fit(X_train, y_train)